# Prepare data for CellXGene

Store datasets in a sparse dataframe, add to AnnData object

To launch:

source activate scanpy

cellxgene launch ./biccn_mop6.h5ad -var gene -v -p 7006 --open

In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
import scipy.sparse as sparse
import scipy.stats as stats
from importlib import reload

import matplotlib.pyplot as plt
import re
import time
import os

from multiprocessing import Pool

In [2]:
# Load per-cell gene data
datadir = '/cndd/emukamel/BICCN-Mouse-MOp/data_freeze_formatted'
modalities = [
#     '10x_cells_v2',
#     '10x_cells_v3',
#     '10x_nuclei_v3_macosko',
#     '10x_nuclei_v3',
    'smarter_cells',
    'smarter_nuclei',
    'snatac_gene',
    'snmcseq_gene',
]
modalities_alt = [
#     '10X_cells_v2_AIBS', '10X_cells_v3_AIBS', 
#     '10X_nuclei_v3_Broad','10X_nuclei_v3_AIBS',
    'SmartSeq_cells_AIBS', 
    'SmartSeq_nuclei_AIBS',
    'Open chromatin',
    'DNA methylation', 
]

In [3]:
# Gather all gene lists
# Load gene names and merge
gencode = pd.read_csv('genes.gencode.vM16.bed',sep='\t',header=None,
                     names=['chrom','start','end','ensid','gene'])
patt = re.compile(r'\..+')
gencode.ensid = [patt.sub('',g) for g in gencode.ensid]
gencode = gencode.set_index('ensid',drop=False)

genes = pd.read_csv(datadir+'/'+modalities[0]+'_raw.gene',header=None)
genes.columns = ['ensID']
genes = genes.set_index('ensID',drop=False)
ngenes = genes.shape[0]

# Load cells
cells = pd.read_csv('/cndd/emukamel/BICCN-Mouse-MOp/MOp_metadata/Table_S2_CellClusterAssignments_AllMethods.tsv',
                   sep='\t',
                    dtype='str')
for mod in ['10X_cells_v2_AIBS', '10X_cells_v3_AIBS','10X_nuclei_v3_AIBS']:
    print(mod)
    curr_rows = cells.modality==mod
    cu=cells.loc[curr_rows,'cell_id']
    cu = [c[:-18] for c in cu]
    print(cu[:5])
    cells.loc[curr_rows,'cell_id'] = cu
cells.head()

10X_cells_v2_AIBS
['10x_cells_v2_AAACCTGAGGAGTCTG-1', '10x_cells_v2_ACCTTTAGTACAGCAG-1', '10x_cells_v2_ACGATACCACCCAGTG-1', '10x_cells_v2_ACGGGTCAGTGGGCTA-1', '10x_cells_v2_AGCAGCCCAGTTCATG-1']
10X_cells_v3_AIBS
['10x_cells_v3_AAACCCAAGCTTCATG-1', '10x_cells_v3_AAACCCACACCAGCCA-1', '10x_cells_v3_AAACGAACAACGATTC-1', '10x_cells_v3_AAACGAATCTCGTGAA-1', '10x_cells_v3_AAACGCTGTAGTCACT-1']
10X_nuclei_v3_AIBS
['10x_nuclei_v3_AAACCCAAGCTCTTCC-1', '10x_nuclei_v3_AAAGGGCTCGCGATCG-1', '10x_nuclei_v3_AAAGTGACATCGCTGG-1', '10x_nuclei_v3_AACAACCGTGCTCTCT-1', '10x_nuclei_v3_AACAAGAAGCCTGCCA-1']


,cell_id,SCF_round2,SCF_round3,RNA concensus,LIGER_level1,LIGER_level2,modality
0,snmcseq_gene_2C_M_0,1-1,1-1-1,NaN,0,it_8,DNA methylation
1,snmcseq_gene_2C_M_1,4-2,4-2-1,NaN,1,it_0,DNA methylation
2,snmcseq_gene_2C_M_100,4-1,4-1-1,NaN,1,it_0,DNA methylation
3,snmcseq_gene_2C_M_1000,4-2,4-2-1,NaN,1,it_0,DNA methylation
4,snmcseq_gene_2C_M_1001,4-1,4-1-1,NaN,1,it_0,DNA methylation


In [6]:
cells

,cell_id,SCF_round2,SCF_round3,RNA concensus,LIGER_level1,LIGER_level2,modality
0,snmcseq_gene_2C_M_0,1-1,1-1-1,NaN,0,it_8,DNA methylation
1,snmcseq_gene_2C_M_1,4-2,4-2-1,NaN,1,it_0,DNA methylation
2,snmcseq_gene_2C_M_100,4-1,4-1-1,NaN,1,it_0,DNA methylation
3,snmcseq_gene_2C_M_1000,4-2,4-2-1,NaN,1,it_0,DNA methylation
4,snmcseq_gene_2C_M_1001,4-1,4-1-1,NaN,1,it_0,DNA methylation
...,...,...,...,...,...,...,...
571698,snatac_gene_CEMBA180618_5D_TCCGGAGAAACCAGGTGTA...,NaN,NaN,NaN,glia,non_neuron_10,Open chromatin
571699,snatac_gene_CEMBA180618_5D_TCCGGAGAACGTTCGAGGT...,NaN,NaN,NaN,glia,non_neuron_10,Open chromatin
571700,snatac_gene_CEMBA180618_5D_TCCGGAGAACGTTCGAGTA...,NaN,NaN,NaN,glia,non_neuron_10,Open chromatin
571701,snatac_gene_CEMBA180618_5D_TCCGGAGACGAGGCTGAGG...,NaN,NaN,NaN,glia,non_neuron_10,Open chromatin


In [25]:
# Prepare the DNA methylation data

# if not os.path.exists(datadir+'/snmcseq_gene_raw.npz'):
    # Prepare the snmcseq data
    ch = sparse.load_npz(datadir+'/snmcseq_gene_CH_raw.npz')
    mch = sparse.load_npz(datadir+'/snmcseq_gene_mCH_raw.npz')

    ch_df = pd.DataFrame.sparse.from_spmatrix(ch).astype(pd.SparseDtype("float", 0))
    mch_df = pd.DataFrame.sparse.from_spmatrix(mch).astype(pd.SparseDtype("float", 0))

    mlevel = mch_df/ch_df
    sparse.save_npz(datadir+'/snmcseq_gene_raw.npz', mlevel)

    mlevel_z = stats.zscore(mlevel,axis=1,nan_policy='omit')
    np.save(datadir+'/snmcseq_gene_raw.zscore.npy', mlevel_z)

In [13]:
def vars(a, axis=None):
    """ Variance of sparse matrix a
    var = mean(a**2) - mean(a)**2
    """
    a_squared = a.copy()
    a_squared.data **= 2
    return a_squared.mean(axis) - np.square(a.mean(axis))

In [14]:
# For each dataset, determine Highly variable genes (HVGs)
mod = 'smarter_cells'
data1 = sparse.load_npz(datadir+'/'+mod+'_raw.npz')

# Load genes
genes1 = pd.read_csv(datadir+'/'+mod+'_raw.gene',header=None)

data1=data1.tocsr()
dv = vars(data1,axis=1)

hvg = dv>np.percentile(dv,80) # Keep the top 20% of most variable genes
hvg_ensid = genes1[hvg]
hvg_ensid.shape

(6465, 1)

In [28]:
def load_df(mod, ensid=None):
    tstart = time.time()
    print('Loading %s' % mod)
    
    # Load genes
    genes1 = pd.read_csv(datadir+'/'+mod+'_raw.gene',header=None)

    # Load data
    if 'snmcseq' in mod:
        data1 = np.load(datadir+'/'+mod+'_raw.zscore.npy').T
#         data1 = sparse.csr_matrix(data1)
#         data1df = pd.SparseDataFrame(data=data1, columns=genes1[0])
        data1df = pd.DataFrame(data1, columns=genes1[0])
        data1df = data1df.astype(pd.SparseDtype("float",fill_value=np.nan))
    else:
        data1 = sparse.load_npz(datadir+'/'+mod+'_raw.npz').T
        data1 = stats.zscore(data1,nan_policy='omit',axis=0)
        data1df = pd.DataFrame.sparse.from_spmatrix(data1, columns = genes1[0])
    
    if ensid is not None:
#         data1df = data1df.filter(axis=1,items=ensid.to_numpy().ravel())
        data1df = data1df.loc[:,ensid.to_numpy().ravel()]


    # Load cells
    cells1 = pd.read_csv(datadir+'/'+mod+'_raw.cell',header=None)
    cells1 = cells1[0]
    cells1 = [mod+'_'+c for c in cells1]
    data1df['cells'] = cells1
    data1df.set_index('cells', inplace=True, drop=True)
    print('Modality %s: t=%3.3f s; %d cells x %d genes' % (mod, time.time()-tstart,
                                                          data1df.shape[0], data1df.shape[1]))
    
    return data1df

def load_metadata(mod):
    tstart = time.time()
    print('Loading metadata for %s' % mod)
    
    # Load data
    data1 = pd.read_csv(datadir+'/'+mod+'_metadata.tsv',sep='\t',
                       columns=['cell','tsne_0','tsne_1'])
    
    return data1

In [ ]:
# For each modality, load the data, subset the genes, and normalize
mod = modalities[3]
mod
tstart = time.time()
df = load_df(mod, ensid=hvg_ensid)
print(time.time() - tstart)

Loading snmcseq_gene


/cndd/emukamel/conda_envs/scanpy/lib/python3.7/site-packages/pandas/core/indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [54]:
# For each modality, load the data, subset the genes, and normalize
mod = modalities[0]
mod

data1 = sparse.load_npz(datadir+'/'+mod+'_raw.npz').T
data1 = stats.zscore(data1,nan_policy='omit',axis=0)

# tstart = time.time()
# df = load_df(mod, ensid=hvg_ensid)
# print(time.time() - tstart)

IndexError: tuple index out of range

In [68]:
# data1z=stats.zscore(data1.todense(),axis=0,nan_policy='omit')
# data1
# x=data1.todense()
# stats.zscore(x)
stats.zscore(x[:,:100],axis=0)

TypeError: mean() got an unexpected keyword argument 'keepdims'

In [52]:
dfs = []
# with Pool() as p:
#     for d in p.map(load_df, modalities):
#         dfs.append(d)
        
for mod in modalities:
    dfs.append(load_df(mod, ensid=hvg_ensid))

Loading smarter_cells


/cndd/emukamel/conda_envs/cembadata_eran/lib/python3.6/site-packages/scipy/stats/stats.py:269: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


IndexError: tuple index out of range

In [ ]:
df = pd.concat(dfs, sort=True)

In [233]:
df = df.drop('cells',axis=1)

In [27]:
# Load tSNE for all datasets
fn='/cndd/Public_Datasets/BICCN/BICCN_minibrain_data/data_freeze/supp_info/clusters_final/miniatlas_fig4_scf_clusterings.tsv'
scf = pd.read_csv(fn,sep='\t',low_memory=False)
scf.set_index('sample',inplace=True,drop=False)
scf = scf.loc[df.index,['sample','modality','modality_name','joint_cluster_round2','joint_cluster_round3',
                       'joint_embedding_x','joint_embedding_y']]
scf.shape

NameError: name 'df' is not defined

In [274]:
gencodeu = gencode.loc[df.columns,['ensid','gene']]

,ensid,gene
ENSMUSG00000000001,ENSMUSG00000000001,Gnai3
ENSMUSG00000000056,ENSMUSG00000000056,Narf
ENSMUSG00000000058,ENSMUSG00000000058,Cav2
ENSMUSG00000000078,ENSMUSG00000000078,Klf6
ENSMUSG00000000085,ENSMUSG00000000085,Scmh1
...,...,...
ENSMUSG00000111375,ENSMUSG00000111375,A830010M20Rik
ENSMUSG00000111765,ENSMUSG00000111765,Gm10635
ENSMUSG00000113429,ENSMUSG00000113429,Gm20063
ENSMUSG00000113450,ENSMUSG00000113450,Zfp935


In [184]:
nuse = 1000
dfu = df.iloc[:nuse,:100]

In [185]:
adata = ad.AnnData(X = dfu,
                   obsm = {'X_tsne':  scf.loc[:,['joint_embedding_x', 'joint_embedding_y']][:nuse].to_numpy()},
                   obs = scf.iloc[:nuse,:],
                   var = gencodeu[:100],
                  );

In [186]:
adata.obs_names_make_unique()
adata.var_names_make_unique()
results_file = './biccn_mop6.test.h5ad'  # the file that will store the analysis results
adata.write_h5ad(results_file)

... storing 'sample' as categorical
... storing 'modality' as categorical
... storing 'modality_name' as categorical
... storing 'joint_cluster_round2' as categorical
... storing 'joint_cluster_round3' as categorical


In [339]:
# Load tSNE for all datasets
samples_use = np.intersect1d(scf['sample'],df.index)
fn='/cndd/Public_Datasets/BICCN/BICCN_minibrain_data/data_freeze/supp_info/clusters_final/miniatlas_fig4_scf_clusterings.tsv'
scf = pd.read_csv(fn,sep='\t',low_memory=False)
scf.set_index('sample',inplace=True,drop=False)
# scf = scf.loc[df.index,:]
scf = scf.loc[samples_use,['sample','modality','modality_name','joint_cluster_round2','joint_cluster_round3',
                       'joint_embedding_x','joint_embedding_y']]

print(scf.shape,dfu.shape)

(70454, 7) (70454, 6465)


In [342]:
dfu = df.loc[samples_use,:]

array([ True,  True,  True, ...,  True,  True,  True])

In [343]:
adata = ad.AnnData(X = dfu,
                   obsm = {'X_tsne':  scf.loc[:,['joint_embedding_x', 'joint_embedding_y']].to_numpy()},
                   obs = scf,
                   var = gencodeu,
                  );

In [344]:
adata.obs_names_make_unique()
adata.var_names_make_unique()
results_file = './biccn_mop6.h5ad'  # the file that will store the analysis results
adata.write_h5ad(results_file)

... storing 'modality' as categorical
... storing 'modality_name' as categorical
... storing 'joint_cluster_round2' as categorical
... storing 'joint_cluster_round3' as categorical
